<a href="https://colab.research.google.com/github/haruyoshikawabe/accounting-automation-python/blob/main/%E4%BC%9A%E8%A8%88%E5%8B%98%E5%AE%9A%E7%A7%91%E7%9B%AE%E6%A4%9C%E7%B4%A2%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab用会計勘定科目検索アプリ (Gradio 5.50.0対応)
# 使用方法: このコードをGoogle Colabのセルにコピー&ペーストして実行

!pip install gradio==5.50.0 -q

import gradio as gr
import pandas as pd

class AccountingApp:
    def __init__(self):
        self.accounts = self.load_accounts()
        self.favorites = []
        self.selected_code = None

    def load_accounts(self):
        """勘定科目データの読み込み"""
        return [
            {"code": "101", "name": "現金", "category": "資産", "description": "手元現金"},
            {"code": "102", "name": "当座預金", "category": "資産", "description": "当座預金口座の残高"},
            {"code": "103", "name": "普通預金", "category": "資産", "description": "普通預金口座の残高"},
            {"code": "111", "name": "受取手形", "category": "資産", "description": "営業上の債権として受け取った手形"},
            {"code": "112", "name": "売掛金", "category": "資産", "description": "商品・サービス提供後の未回収代金"},
            {"code": "121", "name": "商品", "category": "資産", "description": "販売目的の商品在庫"},
            {"code": "122", "name": "製品", "category": "資産", "description": "製造した製品在庫"},
            {"code": "123", "name": "原材料", "category": "資産", "description": "製造用の原材料"},
            {"code": "131", "name": "建物", "category": "資産", "description": "所有する建物"},
            {"code": "132", "name": "備品", "category": "資産", "description": "事務用品、什器など"},
            {"code": "133", "name": "車両運搬具", "category": "資産", "description": "業務用車両"},
            {"code": "141", "name": "ソフトウェア", "category": "資産", "description": "購入または開発したソフトウェア"},

            {"code": "201", "name": "支払手形", "category": "負債", "description": "仕入等で振り出した手形"},
            {"code": "202", "name": "買掛金", "category": "負債", "description": "商品・サービス購入後の未払金"},
            {"code": "211", "name": "短期借入金", "category": "負債", "description": "1年以内返済予定の借入金"},
            {"code": "212", "name": "長期借入金", "category": "負債", "description": "1年超の返済期限の借入金"},
            {"code": "221", "name": "未払金", "category": "負債", "description": "営業外取引の未払金"},
            {"code": "222", "name": "未払費用", "category": "負債", "description": "継続的サービスの未払分"},
            {"code": "231", "name": "預り金", "category": "負債", "description": "源泉税、社会保険料などの預り金"},

            {"code": "301", "name": "資本金", "category": "純資産", "description": "出資者からの出資額"},
            {"code": "302", "name": "利益剰余金", "category": "純資産", "description": "過去からの蓄積利益"},

            {"code": "401", "name": "売上高", "category": "収益", "description": "商品・サービスの販売収入"},
            {"code": "411", "name": "受取利息", "category": "収益", "description": "預金等の利息収入"},
            {"code": "412", "name": "受取配当金", "category": "収益", "description": "株式等の配当収入"},
            {"code": "421", "name": "雑収入", "category": "収益", "description": "その他の収入"},

            {"code": "501", "name": "仕入高", "category": "費用", "description": "商品の仕入原価"},
            {"code": "511", "name": "給料手当", "category": "費用", "description": "従業員への給与"},
            {"code": "512", "name": "法定福利費", "category": "費用", "description": "社会保険料の会社負担分"},
            {"code": "521", "name": "地代家賃", "category": "費用", "description": "事務所・店舗等の賃借料"},
            {"code": "522", "name": "水道光熱費", "category": "費用", "description": "電気・ガス・水道料金"},
            {"code": "531", "name": "旅費交通費", "category": "費用", "description": "出張費、交通費"},
            {"code": "532", "name": "通信費", "category": "費用", "description": "電話・インターネット料金"},
            {"code": "533", "name": "消耗品費", "category": "費用", "description": "文房具等の消耗品"},
            {"code": "541", "name": "広告宣伝費", "category": "費用", "description": "広告・宣伝のための費用"},
            {"code": "542", "name": "接待交際費", "category": "費用", "description": "取引先との接待費用"},
            {"code": "551", "name": "支払利息", "category": "費用", "description": "借入金等の利息"},
            {"code": "561", "name": "減価償却費", "category": "費用", "description": "固定資産の減価償却"},
            {"code": "571", "name": "租税公課", "category": "費用", "description": "税金・公的負担金"},
            {"code": "581", "name": "雑費", "category": "費用", "description": "その他の費用"},
        ]

    def search_accounts(self, keyword, category, show_favorites):
        """勘定科目の検索"""
        keyword = keyword.lower() if keyword else ""

        filtered_accounts = []
        for account in self.accounts:
            # お気に入りフィルター
            if show_favorites and account["code"] not in self.favorites:
                continue

            # カテゴリーフィルター
            if category != "全て" and account["category"] != category:
                continue

            # キーワード検索
            if keyword:
                if (keyword not in account["name"].lower() and
                    keyword not in account["code"].lower() and
                    keyword not in account["description"].lower()):
                    continue

            filtered_accounts.append([
                account["code"],
                account["name"],
                account["category"],
                "★" if account["code"] in self.favorites else ""
            ])

        if not filtered_accounts:
            return pd.DataFrame(columns=["コード", "勘定科目名", "カテゴリー", "お気に入り"])

        return pd.DataFrame(filtered_accounts, columns=["コード", "勘定科目名", "カテゴリー", "お気に入り"])

    def show_detail_by_code(self, code_input):
        """コード入力による詳細表示"""
        if not code_input:
            return "コードを入力してください"

        account = next((a for a in self.accounts if a["code"] == code_input), None)
        if not account:
            return f"コード {code_input} の勘定科目が見つかりません"

        self.selected_code = code_input
        return self.format_detail(account)

    def show_detail_by_selection(self, selected_index):
        """ドロップダウン選択による詳細表示"""
        if not selected_index:
            return "勘定科目を選択してください"

        # "コード - 科目名" から科目名を抽出
        parts = selected_index.split(" - ")
        if len(parts) < 2:
            return "選択が無効です"

        code = parts[0]
        account = next((a for a in self.accounts if a["code"] == code), None)

        if not account:
            return "勘定科目が見つかりません"

        self.selected_code = code
        return self.format_detail(account)

    def format_detail(self, account):
        """詳細情報のフォーマット"""
        detail = f"""## {account['name']}

**コード:** {account['code']}

**カテゴリー:** {account['category']}

**説明:**
{account['description']}

---

### 仕訳例

"""

        # カテゴリー別の仕訳例
        if account['category'] == '資産':
            detail += f"{account['name']}が増加した場合:  \n(借方) {account['name']} xxx / (貸方) ･･･ xxx"
        elif account['category'] == '負債':
            detail += f"{account['name']}が増加した場合:  \n(借方) ･･･ xxx / (貸方) {account['name']} xxx"
        elif account['category'] == '純資産':
            detail += f"{account['name']}が増加した場合:  \n(借方) ･･･ xxx / (貸方) {account['name']} xxx"
        elif account['category'] == '収益':
            detail += f"{account['name']}が発生した場合:  \n(借方) 現金等 xxx / (貸方) {account['name']} xxx"
        elif account['category'] == '費用':
            detail += f"{account['name']}が発生した場合:  \n(借方) {account['name']} xxx / (貸方) 現金等 xxx"

        return detail

    def toggle_favorite(self):
        """お気に入りの追加/削除"""
        if not self.selected_code:
            return "勘定科目を選択してください", self.get_account_choices()

        if self.selected_code in self.favorites:
            self.favorites.remove(self.selected_code)
            message = f"コード {self.selected_code} をお気に入りから削除しました"
        else:
            self.favorites.append(self.selected_code)
            message = f"コード {self.selected_code} をお気に入りに追加しました"

        # 選択肢を更新
        return message, self.get_account_choices()

    def get_account_choices(self):
        """ドロップダウンの選択肢を生成"""
        choices = []
        for account in self.accounts:
            fav_mark = "★ " if account["code"] in self.favorites else ""
            choices.append(f"{fav_mark}{account['code']} - {account['name']} ({account['category']})")
        return choices

# アプリのインスタンス作成
app = AccountingApp()

# Gradio UIの構築
with gr.Blocks(title="会計勘定科目検索アプリ", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📊 会計勘定科目検索アプリ")
    gr.Markdown("勘定科目を検索して、詳細情報や仕訳例を確認できます")

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 🔍 検索条件")
            with gr.Row():
                keyword_input = gr.Textbox(
                    label="キーワード",
                    placeholder="勘定科目名、コード、説明で検索...",
                    scale=2
                )
                category_dropdown = gr.Dropdown(
                    choices=["全て", "資産", "負債", "純資産", "収益", "費用"],
                    value="全て",
                    label="カテゴリー",
                    scale=1
                )
                favorites_checkbox = gr.Checkbox(
                    label="お気に入りのみ",
                    value=False
                )

            search_button = gr.Button("🔍 検索", variant="primary")

            gr.Markdown("### 📋 検索結果")
            results_table = gr.Dataframe(
                headers=["コード", "勘定科目名", "カテゴリー", "お気に入り"],
                interactive=False,
                wrap=True
            )

            gr.Markdown("### 📌 勘定科目を選択")
            with gr.Row():
                account_selector = gr.Dropdown(
                    choices=app.get_account_choices(),
                    label="勘定科目一覧",
                    scale=3
                )
                code_input = gr.Textbox(
                    label="またはコード入力",
                    placeholder="例: 101",
                    scale=1
                )

        with gr.Column(scale=1):
            gr.Markdown("### 📝 詳細情報")
            detail_output = gr.Markdown("勘定科目を選択してください")

            gr.Markdown("---")
            favorite_button = gr.Button("⭐ お気に入り追加/削除", variant="secondary")
            favorite_status = gr.Textbox(label="ステータス", interactive=False)

    # イベントハンドラ
    search_button.click(
        fn=app.search_accounts,
        inputs=[keyword_input, category_dropdown, favorites_checkbox],
        outputs=results_table
    )

    # キーワード入力時にも自動検索
    keyword_input.change(
        fn=app.search_accounts,
        inputs=[keyword_input, category_dropdown, favorites_checkbox],
        outputs=results_table
    )

    category_dropdown.change(
        fn=app.search_accounts,
        inputs=[keyword_input, category_dropdown, favorites_checkbox],
        outputs=results_table
    )

    favorites_checkbox.change(
        fn=app.search_accounts,
        inputs=[keyword_input, category_dropdown, favorites_checkbox],
        outputs=results_table
    )

    # ドロップダウン選択時の詳細表示
    account_selector.change(
        fn=app.show_detail_by_selection,
        inputs=account_selector,
        outputs=detail_output
    )

    # コード入力時の詳細表示
    code_input.submit(
        fn=app.show_detail_by_code,
        inputs=code_input,
        outputs=detail_output
    )

    # お気に入りボタン
    def update_favorite_and_choices():
        message, choices = app.toggle_favorite()
        return message, gr.Dropdown(choices=choices)

    favorite_button.click(
        fn=app.toggle_favorite,
        outputs=[favorite_status, account_selector]
    )

    # 初期表示
    demo.load(
        fn=app.search_accounts,
        inputs=[keyword_input, category_dropdown, favorites_checkbox],
        outputs=results_table
    )

# アプリの起動
demo.launch(share=True, debug=True)

/tmp/ipython-input-349001622.py:185: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="会計勘定科目検索アプリ", theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://070d07eac2862ce633.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
